In [1]:
!pip install yfinance
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly

In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Histórico del precio de la acción", "Histórico de ingresos"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Fecha", row=1, col=1)
    fig.update_xaxes(title_text="Fecha", row=2, col=1)
    fig.update_yaxes(title_text="Precio ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Ingresos ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,height=900,title=stock,xaxis_rangeslider_visible=True)
    fig.show()

In [4]:

GameStop = yf.Ticker("GME")

In [5]:
datos_gme=GameStop.history(period="max")
datos_gme

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
2002-02-19 00:00:00-05:00,1.666417,1.666417,1.578047,1.607504,7410400,0.0,0.0
2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603296,1.662209,6892800,0.0,0.0
...,...,...,...,...,...,...,...
2024-09-30 00:00:00-04:00,22.299999,23.639999,22.170000,22.930000,10161700,0.0,0.0
2024-10-01 00:00:00-04:00,22.950001,23.120001,21.809999,21.969999,7189200,0.0,0.0
2024-10-02 00:00:00-04:00,21.860001,22.299999,21.410000,21.490000,5673100,0.0,0.0


In [6]:
datos_gme.reset_index(inplace=True)
datos_gme.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666417,1.666417,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603296,1.662209,6892800,0.0,0.0


In [7]:
import pandas as pd
from bs4 import BeautifulSoup

# Cargar el HTML guardado
with open('gamestop.html', 'r', encoding='utf-8') as file:
    datos_html = file.read()

# Analizar el HTML con BeautifulSoup
beautiful_soup = BeautifulSoup(datos_html, "html5lib")

# Encontrar todas las tablas en el HTML
tablas = beautiful_soup.find_all('table')

# Buscar la tabla "GameStop Quarterly Revenue"
for indice, tabla in enumerate(tablas):
    if "GameStop Quarterly Revenue" in str(tabla):
        indice_tabla = indice
        break

# Crear una lista para almacenar los datos de ingresos
ingresos_list = []

# Extraer los datos de la tabla
for linea in tablas[indice_tabla].tbody.find_all("tr"):
    col = linea.find_all("td")
    if col:
        fecha = col[0].text.strip()
        ingresos = col[1].text.replace("$", "").replace(",", "").strip()
        ingresos_list.append({"Date": fecha, "Revenue": ingresos})

# Crear el DataFrame a partir de la lista
ingresos_gme = pd.DataFrame(ingresos_list)

# Eliminar filas con ingresos vacíos
ingresos_gme = ingresos_gme[ingresos_gme['Revenue'] != ""]

# Convertir la columna 'Revenue' a tipo float
ingresos_gme['Revenue'] = ingresos_gme['Revenue'].astype(float)

# Mostrar las últimas 5 filas del DataFrame
print(ingresos_gme.tail())

          Date  Revenue
58  2010-01-31   3524.0
59  2009-10-31   1835.0
60  2009-07-31   1739.0
61  2009-04-30   1981.0
62  2009-01-31   3492.0


In [8]:
ingresos_gme.tail()

,Date,Revenue
58,2010-01-31,3524.0
59,2009-10-31,1835.0
60,2009-07-31,1739.0
61,2009-04-30,1981.0
62,2009-01-31,3492.0
